!["question"](question.jpg)



between layer 1 and 2:

$$
\begin{bmatrix}
  b_{10} & w_{11} & w_{21} \\
  b_{20} & w_{12} & w_{22} 
\end{bmatrix}
$$

where b = bias and w = weights

$x = [0.1,0.2]$

In [1]:
import numpy as np
# from random import seed
from sklearn.datasets import load_iris


In [41]:
data = load_iris()
target = data.target[(data.target == 0) | (data.target == 1)]
data = data.data[(data.target == 0) | (data.target == 1)]


((100,), (100, 4))

input is columns, output is rows 

In [2]:
def Linear(num_nodes_input_layer, num_nodes_output_layer):
    """Dimensions: output is rows, input is columns."""
    weights = np.random.rand(num_nodes_output_layer, num_nodes_input_layer + 1) #* 0.01
    # biases = np.ones((weights.shape[0], 1))

    # Stack the original array and the column of ones
    # return np.column_stack((biases, weights ))
    return weights


# Forward propogation to make predictions

!["how to apply the weights"](apply_weights.png)

$$ output = 
\begin{bmatrix}
  Neuron\:1\:applied\:to\:sample\:1 & Neuron\:2\:applied\:to\:sample\:1 \\
  Neuron\:1\:applied\:to\:sample\:2 & Neuron\:2\:applied\:to\:sample\:2
\end{bmatrix}
$$

In [3]:
def compute_forward(inputs, layer):
    ## linear combination w0x0 + w1x1 + w2x2
    linear_combinations = []
    # print(inputs, inputs.shape)
    # print(np.ones(inputs.shape[1]))
    inputs = np.c_[ np.ones(inputs.shape[0]), inputs  ]    ## add bias to the inputs
    # print(inputs)
    # print(layer)

    linear_combinations = np.matmul(inputs, layer.T) ## logits or z
    
    # print(linear_combinations)

    sigmoids = 1/(  1 + np.exp( -linear_combinations ) ) ## compute a
    # print(sigmoids)

    return linear_combinations, sigmoids


# def sigmoid(inputs, layer):

#     linear_combinations = np.matmul(inputs, layer.T) ## logits
#     return 1/(  1 + np.exp( -linear_combinations ) ) ## compute a

# Loss function

*m is number of samples*

*k is total number of output units*
$$
 J(θ)= - \frac{1}{m} {\sum^m_{i=1}\sum^k_{k=1} [y^{(i)}_kln(\frac{1}{1+e^{xθ^T}})_k + (1-y^{(i)}_k)ln(1-(\frac{1}{1+e^{xθ^T}})_k)]} 
$$
$$
 = \frac{1}{m} {\sum^m_{i=1}\sum^k_{k=1} [y^{(i)}_kln(a^{(L)}_k + (1-y^{(i)}_k)ln(1-a^{(L)}_k)]}

$$


$$  
output\_sigmoids = 
\begin{bmatrix}
  Neuron\:1\:applied\:to\:sample\:1 & Neuron\:2\:applied\:to\:sample\:1 \\
  Neuron\:1\:applied\:to\:sample\:2 & Neuron\:2\:applied\:to\:sample\:2
\end{bmatrix}
$$

In [4]:
def binary_cross_entropy_loss(output_sigmoids, target):
    
    ## m is number of samples
    ## k is total number of output units

    target = target.reshape(-1,1)
    log_sigmoids = np.log(output_sigmoids)
    y_times_log_sigmoids = np.matmul(log_sigmoids, target.T)
    # print(y_times_log_sigmoids)

    one_minus_log_sigmoids = 1 - np.log(output_sigmoids)
    one_minus_y_times_log_sigmoids = np.matmul(one_minus_log_sigmoids, (1 - target).T)
    # print(one_minus_y_times_log_sigmoids)
    
    return sum(sum((y_times_log_sigmoids + one_minus_y_times_log_sigmoids)))/len(target) ## sum across the output units and sum across the samples
    # for sample in target:
    #     for row in output_sigmoids:
            
    #         print(row, sample)
    #     print("_____________")




# Backpropogation

want: $\frac{∂J(θ)}{∂θ^{(l)}_{ij}}$ for $l=1,2$

for each

$$δ^{(L)} = \begin{bmatrix}
  a^{(L)}_{1} - y_{11} \\
  a^{(L)}_{2} - y_{12} \\
  a^{(L)}_1 - y_{21} \\ 
  a^{(L)}_1 - y_{22} \\ 

\end{bmatrix}
$$

$y_{11}$ is sample $1$, for target $1 $

$$δ^{(L)} * \begin{bmatrix} a^{(L-1)}_{0}  a^{(L-1)}_{1} a^{(L-1)}_2 \end{bmatrix} = \begin{bmatrix}
  \frac{∂J(θ)}{∂θ^{(L-1)}_{10}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{11}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{12}} \\
  \\
  \frac{∂J(θ)}{∂θ^{(L-1)}_{20}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{21}} & \frac{∂J(θ)}{∂θ^{(L-1)}_{22}} \\


\end{bmatrix} $$

slides 37, 50

In [380]:
x = np.array([[0.1,0.2],
            [0.3,0.8], 
            [0.3,0.8]]) # x1, x2

y = np.array([[1,0],[1,1],[0,1]])

In [ ]:
## define the layers
# seed(1)
#input = Linear(3,2)
seed(1)
hidden_layer1 = Linear(2,2)
seed(1)
output_layer = Linear(2,1)

# hidden_layer1

In [393]:
# sigmoids2, a2 = compute_forward(x, input)

# sigmoids3, a3 = compute_forward(a2, hidden_layer1)

z2, sigmoids2 = compute_forward(x, hidden_layer1)

z_output, output_sigmoids = compute_forward(z2, output_layer)


In [401]:
binary_cross_entropy_loss(output_sigmoids, y)

1.6995779071347037

In [11]:
def compute_backpropogation(sigmoids: list, layers: list, y, x):
    """"""
    # print("layers before reversing")
    # print(layers)
    # layers = [hidden_layer1, output_layer]

    layers.reverse()
    # print("layers after reversing")
    # print(layers)
    # sigmoids = [sigmoids2, output_sigmoids]

    sigmoids.insert(0, x)     
    print("sigmoids before reversing")
    print(sigmoids)
    sigmoids.reverse()
    # print("sigmoids after reversing")
    # print(sigmoids)
    gradients = []

    y = y.reshape(-1,1)
    # print(y)

    for i, layer in enumerate(layers):
        if i == 0:

            delta = (sigmoids[i] - y)
            # print(delta.shape)
            previous_sigmoids = np.c_[ np.ones(sigmoids[i+1].shape[0]), sigmoids[i+1]  ]    ## add bias to the inputs
            # print("sigmoids for the last layer")
            # print(sigmoids[i])
            # # print("__ __ __ __")
            # print("sigmoids for the previous layer")
            # print(previous_sigmoids)

            ## delta has shape (samples x nodes)
            ## sigmoids has shape (samples x activations + 1)
            ## ex delta = sx2 (s = samples & nodes = 2 )
            ##    sigmoids = sx3 (s = samples & there are 2 sigmoids and 1 to multiple the bias by)
            gradients.append(np.matmul(delta.T, previous_sigmoids))


        else:
            ## the gradients accumulate across the samples so I may need to do
            ## this for each sample then add all the gradients together. 
            # print("_______________________________")
            ## after the first delta, every previous delta is
            ## (delta * (weights from the previous layer)) * sigmoids or x
            delta = np.matmul(delta, layers[i-1][:,1:]) * sigmoids[i] * (1- sigmoids[i])
            previous_sigmoids = np.c_[ np.ones(sigmoids[i+1].shape[0]), sigmoids[i+1]  ]    ## add bias to the inputs
            # print(previous_sigmoids)
            gradients.append(np.matmul(delta.T, previous_sigmoids))


    # print(gradients)
    return gradients

def step(weights: list, gradients: list, lr = 0.01):
    updated_weights = []
    gradients.reverse()
    for weight, gradient in zip(weights, gradients): ## so the first gradient is the input layers graidents
        yield weight - (gradient * lr)
        # weight = weight + (gradient * lr)
        # updated_weights.append(weight)

    # return updated_weights
    


In [112]:
# import numpy as np

def compute_backpropogation(sigmoids: list, layers: list, y: np.ndarray, x: np.ndarray):
    """
    sigmoids: [A1, A2, ..., A_L]  (activations for each hidden and output layer, no bias columns)
    layers:   [W1, W2, ..., W_L]  (Wk shape: (nodes_out_k, nodes_in_k + 1))
    x:        input matrix (N, input_dim)
    y:        target (N, 1)
    returns:  gradients list in same order as layers: [dW1, dW2, ..., dW_L]
    """

    # Make local copies — do not reverse/mutate the originals.
    W = [w.copy() for w in layers]
    A = [a.copy() for a in sigmoids]

    # Build S = [X, A1, A2, ..., A_L]
    S = [x] + A
    N = x.shape[0]
    y = y.reshape(-1, 1)

    L = len(W)   # number of layers

    grads = [None] * L

    # ---------- OUTPUT LAYER ----------
    A_L = S[-1]             # last activation (N, out_dim)
    A_prev = S[-2]          # activation of previous layer (N, prev_dim)
    # delta for output (sigmoid + BCE): dZ = A_L - y
    delta = (A_L - y)       # (N, out_dim)

    # gradient dW_L = delta^T @ [1, A_prev]
    A_prev_bias = np.c_[np.ones(N), A_prev]          # (N, prev_dim+1)
    dW_L = delta.T @ A_prev_bias                     # (out_dim, prev_dim+1)
    grads[-1] = dW_L

    # ---------- HIDDEN LAYERS (iterate backwards) ----------
    # iterate layer index from L-1 .. 0 (0-based)
    for idx in range(L-2, -1, -1):
        W_next = W[idx+1]            # next layer weights (out_next, in_next+1)
        # remove bias column of next layer to propagate delta
        W_next_no_bias = W_next[:, 1:]   # (out_next, in_next)  (note: out_next corresponds to current layer nodes)

        # delta shape: (N, out_next). Multiply by W_next_no_bias (out_next, in_curr) -> (N, in_curr)
        delta = (delta @ W_next_no_bias) * ( S[idx+1] * (1.0 - S[idx+1]) )  # (N, in_curr) elementwise
        # previous activation (for gradient) is S[idx] (X for idx==0)
        A_prev = S[idx]   # (N, in_curr_prev)
        A_prev_bias = np.c_[np.ones(N), A_prev]   # (N, in_prev+1)
        dW = delta.T @ A_prev_bias               # (in_curr, in_prev+1)  but note sizes align with W[idx] shape
        grads[idx] = dW

    # grads is [dW1, dW2, ..., dW_L] which matches layers order
    return grads

def step(weights: list, gradients: list, lr = 0.001):
    """
    Non-generator updater. Returns updated weights list in same order.
    """
    updated = []
    # gradients already returned in the same order as weights above
    for W, g in zip(weights, gradients):
        updated.append(W + lr * g)
    return updated

# ------------------- Debugging helpers -------------------
def debug_print_state(epoch, z_list, sigmoids_list, grads, weights_before, weights_after):
    print(f"--- epoch {epoch} ---")
    for i, (z, a) in enumerate(zip(z_list, sigmoids_list), 1):
        print(f"Layer {i}: z mean={np.mean(z):.4f}, z std={np.std(z):.4f}, a mean={np.mean(a):.4f}, a std={np.std(a):.4f}")
    for i, g in enumerate(grads, 1):
        print(f"grad L{i} norm: {np.linalg.norm(g):.6f}")
    for i, (wb, wa) in enumerate(zip(weights_before, weights_after), 1):
        print(f"W{i} change norm: {np.linalg.norm(wa - wb):.8f}")

# ------------------- Usage notes -------------------
# 1) Call compute_backpropogation with your sigmoids in forward order [A1, A2, ...]
# 2) Use step(...) to update all weights and reassign them (no generator).
# 3) Use debug_print_state to inspect outputs/gradients/weight changes.


In [12]:
### put it all together:
EPOCHS = 20
x = np.array([[0.1,0.2],
              [-.04,-0.6], 
              [0.1,0.2]]) # x1, x2

# # y = np.array([[1,0],[1,1],[0,1]])
y = np.array([[0],[1],[0]])


hidden_layer1 = Linear(2,2) 

output_layer = Linear(2,1) 

# print(hidden_layer1, output_layer)
for epoch in range(EPOCHS):

    z2, sigmoids2 = compute_forward(x, hidden_layer1)

    z_output, output_sigmoids = compute_forward(sigmoids2, output_layer)

    loss = binary_cross_entropy_loss(output_sigmoids, y)


    # l2_gradients = compute_backpropogation(output_sigmoids, y, sigmoids2)


    # l1_gradients = compute_backpropogation(sigmoids2, y, x)

    # output_layer1 = step(hidden_layer1, l1_gradients, 0.1)

    gradients = compute_backpropogation(sigmoids = [sigmoids2, output_sigmoids], layers = [hidden_layer1, output_layer], y=y, x=x)
    hidden_layer1, output_layer = step(weights = [hidden_layer1, output_layer], gradients = gradients, lr = 0.05)
    # print(gradients)
    

    print(f"----------------------epoch: {epoch}-----------------------")
    print(f"predictions:\n{(output_sigmoids > 0.5).astype(int)}\nActual:\n{y}")
    print(f"error: {loss}")
    print(output_sigmoids)
    

    # update_all_layers([hidden_layer1, output_layer])

sigmoids before reversing
[array([[ 0.1 ,  0.2 ],
       [-0.04, -0.6 ],
       [ 0.1 ,  0.2 ]]), array([[0.64065094, 0.75245143],
       [0.532267  , 0.63693633],
       [0.64065094, 0.75245143]]), array([[0.83696988],
       [0.8132223 ],
       [0.83696988]])]
----------------------epoch: 0-----------------------
predictions:
[[1]
 [1]
 [1]]
Actual:
[[0]
 [1]
 [0]]
error: 2.1875617179683853
[[0.83696988]
 [0.8132223 ]
 [0.83696988]]
sigmoids before reversing
[array([[ 0.1 ,  0.2 ],
       [-0.04, -0.6 ],
       [ 0.1 ,  0.2 ]]), array([[0.63798371, 0.75020661],
       [0.530096  , 0.63487721],
       [0.63798371, 0.75020661]]), array([[0.81519327],
       [0.79111218],
       [0.81519327]])]
----------------------epoch: 1-----------------------
predictions:
[[1]
 [1]
 [1]]
Actual:
[[0]
 [1]
 [0]]
error: 2.214325200993765
[[0.81519327]
 [0.79111218]
 [0.81519327]]
sigmoids before reversing
[array([[ 0.1 ,  0.2 ],
       [-0.04, -0.6 ],
       [ 0.1 ,  0.2 ]]), array([[0.6356089 , 0.7

In [23]:
hidden_layer1

array([[0.37134068, 0.2542515 , 0.29879574],
       [0.34602962, 0.84902082, 0.0763705 ]])

In [21]:
output_sigmoids

array([[0.69629129],
       [0.73525819],
       [0.69629129]])

In [ ]:
def forward(inputs):
    inputs = in

array([[0.13692008, 0.4647963 ],
       [0.65395656, 0.00602726]])

In [46]:
import torch
import torch.nn as nn
import torch.optim as optim

In [49]:
class ANN(nn.Module):
    def __init__(self):
        super().__init__()


        # self.input = nn.Linear(2,2)
        self.hidden_layer1 = nn.Linear(4,2)
        self.output_layer = nn.Linear(2,2)

    def forward(self, x):
        # x = self.input(x)
        x = torch.sigmoid(x)
        x = self.hidden_layer1(x)
        x = torch.sigmoid(x)
        x = self.output_layer(x)

        return x
    
model = ANN()

x_vec = torch.tensor( data )
tensor_target = torch.tensor(target)

# ----- Forward Propagation -----
output = model(x_vec)

# ----- Compute Loss -----
criterion = nn.BCEWithLogitsLoss()
loss = criterion(output, tensor_target)

# ----- Backpropagation -----
optimizer = optim.SGD(model.parameters(), lr = 0.01)
optimizer.zero_grad() ## remove old gradients
loss.backward()
optimizer.step()





RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float